# Issue 139 Notebook Dricksvatten nära SAT 

* denna [Notebook](https://github.com/salgo60/Stockholm_Archipelago_Trail/blob/main/notebook/Issue_139_Notebook_Dricksvatten_n%C3%A4ra_SAT.ipynb)
* [Issue 139](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/139)


Output 
* [kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_2025_08_17_23_13.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_2025_08_17_23_13.html)
* [kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_2025_08_18_19_39.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_2025_08_18_19_39.html)
* [kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_2025_08_18_19_49.html](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_2025_08_18_19_49.html)


version 0.1

In [1]:
import time
from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


Start: 2025-08-18 19:39:34


In [3]:
# !pip install geopandas shapely folium requests SPARQLWrapper --quiet

import os, re, requests
from urllib.parse import quote
import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString, MultiLineString, Point, mapping
from SPARQLWrapper import SPARQLWrapper, JSON
from collections import defaultdict
import folium
from folium import Marker, Icon, FeatureGroup, LayerControl, Popup
from datetime import datetime

# =========================
# 1) Hämta SAT-etapper via Wikidata
# =========================
print("🔍 Hämtar SAT-etapper från Wikidata...")
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?item ?itemLabel ?islandLabel ?osmid WHERE {
  ?item wdt:P361 wd:Q131318799;
        wdt:P31 wd:Q2143825;
        wdt:P402 ?osmid.
  OPTIONAL { ?item wdt:P706 ?island. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

etapper = []
for r in results["results"]["bindings"]:
    etapper.append({
        "id": r["osmid"]["value"],
        "label": r.get("itemLabel", {}).get("value", ""),
        "island": r.get("islandLabel", {}).get("value", "")
    })
osm_ids = [e['id'] for e in etapper]
print(f"✅ Hittade {len(osm_ids)} etapper med OSM-relationer")

# =========================
# 2) Hämta geometrier per relation via Overpass
# =========================
print("📡 Hämtar geometrier från Overpass (per relation)...")
overpass_url = "http://overpass-api.de/api/interpreter"
geom_per_rel = {}
all_lines = []

for rel_id in osm_ids:
    q = f"""
    [out:json][timeout:60];
    relation({rel_id});
    (._;>>;);
    out geom;
    """
    r = requests.post(overpass_url, data={"data": q})
    if r.status_code != 200:
        print(f"⚠️ Fel för relation {rel_id}: {r.text[:200]}...")
        continue
    rel_geoms = []
    for el in r.json().get("elements", []):
        if el.get("type") == "way" and "geometry" in el:
            coords = [(pt["lon"], pt["lat"]) for pt in el["geometry"]]
            if len(coords) >= 2:
                line = LineString(coords)
                rel_geoms.append(line)
                all_lines.append(line)
    if rel_geoms:
        geom_per_rel[rel_id] = rel_geoms

if not all_lines:
    raise ValueError("Inga geometrier hittades från OSM-relationer kopplade via Wikidata.")

# Bygg MultiLineString/LineString per etapp
meta_rows, geom_rows = [], []
for meta in etapper:
    geoms = geom_per_rel.get(meta["id"])
    if not geoms:
        print(f"⚠️ Saknar geometri för {meta['label']} (rel {meta['id']}) – hoppar över.")
        continue
    meta_rows.append(meta)
    geom_rows.append(MultiLineString(geoms) if len(geoms) > 1 else geoms[0])

gdf_trail = gpd.GeoDataFrame(geometry=all_lines, crs="EPSG:4326")
meta_gdf = gpd.GeoDataFrame(meta_rows, geometry=geom_rows, crs="EPSG:4326")
print(f"🧭 Etapper med geometri: {len(meta_gdf)}")

# =========================
# 3) Buffert 200 m (Shapely 2: union_all)
# =========================
print("🧮 Skapar 200 m-buffert...")
buffer_utm = gdf_trail.to_crs(3006).buffer(200)   # 200 m i SWEREF 99 TM
buffer_wgs84 = buffer_utm.to_crs(4326)            # tillbaka till WGS84
buffer_union = buffer_wgs84.union_all()           # EN (multi)polygon

# =========================
# 4) Hämta dricksvatten (noder/ways/relationer) via nwr + out center
# =========================
bbox = gdf_trail.total_bounds  # [minx, miny, maxx, maxy]
q_water = f"""
[out:json][timeout:60];
nwr["amenity"="drinking_water"]({bbox[1]},{bbox[0]},{bbox[3]},{bbox[2]});
out center;
"""
print("💧 Hämtar dricksvatten (nwr) från Overpass...")
r = requests.post(overpass_url, data={"data": q_water})
elements = r.json().get("elements", [])

waters = []
for el in elements:
    tags = el.get("tags", {})
    typ = el.get("type")
    if typ == "node":
        lon, lat = el["lon"], el["lat"]
    else:
        center = el.get("center")
        if not center:
            continue
        lon, lat = center["lon"], center["lat"]
    waters.append({
        "geometry": Point(lon, lat),
        "tags": tags,
        "id": el["id"],
        "osm_type": typ,
        # Dricksvatten saknar normalt "count"-taggar; anta 1 punkt per OSM-objekt
        "water_sites": 1,
    })

gdf_water = gpd.GeoDataFrame(waters, crs="EPSG:4326")
print(f"✅ Hittade {len(gdf_water)} dricksvatten-objekt inom bbox")

# =========================
# 5) Filtrera till de som ligger inom/vid 200 m-bufferten
# =========================
in_range = gdf_water[gdf_water.geometry.covered_by(buffer_union)]
print(f"✅ {len(in_range)} dricksvatten-objekt inom/vid 200 m")

# =========================
# 6) Närmaste etapp per dricksvatten
# =========================
meta_utm = meta_gdf.to_crs(3006)
water_utm = in_range.to_crs(3006)
joined = gpd.sjoin_nearest(
    water_utm,
    meta_utm[["label", "island", "geometry"]],
    how="left",
    distance_col="distance_m"
).to_crs(4326)

# =========================
# 7) Summary 
# =========================
summary = (
    joined.groupby(["label", "island"], as_index=False)
    .agg(
        sites=("geometry", "count"),        # antal OSM-objekt (noder/ways/relationer)
        avg_distance_m=("distance_m", "mean"),
    )
    .sort_values(["sites"], ascending=[False])
)
print("📊 Summary drinking water:")
print(summary.head(1000))

# =========================
# 8) Spara filer + Folium-karta
# =========================
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
os.makedirs("../kartor", exist_ok=True)

# Spara summary
summary_csv = f"../kartor/sat_dricksvatten_summary_{timestamp}.csv"
summary.to_csv(summary_csv, index=False)

# Spara dricksvatten inom 200 m (GeoJSON + CSV)
waters_geojson = f"../kartor/sat_dricksvatten_inrange_{timestamp}.geojson"
waters_csv = f"../kartor/sat_dricksvatten_inrange_{timestamp}.csv"
in_range[["id", "osm_type", "tags", "geometry"]].to_file(waters_geojson, driver="GeoJSON")
in_range.drop(columns="geometry").to_csv(waters_csv, index=False)

# Bygg karta
center = gdf_trail.geometry.union_all().centroid
m = folium.Map(location=[center.y, center.x], zoom_start=9, control_scale=True)

# Etapper
colors = [
    "blue","green","purple","orange","darkred","cadetblue","lightgray","darkblue",
    "darkgreen","pink","lightblue","lightgreen","gray","black","beige","lightred"
]
for i, row in meta_gdf.reset_index(drop=True).iterrows():
    color = colors[i % len(colors)]
    popup = f"<b>{row['label']}</b><br>Ö: {row['island']}"
    folium.GeoJson(
        data=mapping(row.geometry),
        name=row["label"],
        style_function=lambda x, c=color: {"color": c, "weight": 3}
    ).add_child(folium.Popup(popup, max_width=350)).add_to(m)

# Buffert
folium.GeoJson(
    data=mapping(buffer_union),
    name="200 m Buffert",
    style_function=lambda x: {'fillColor': '#0000ff', 'color': '#0000ff', 'weight': 1, 'fillOpacity': 0.1}
).add_to(m)

# Dricksvatten
def _split_multi(s: str):
    """Split tag lists like 'A;B|C' into ['A','B','C']."""
    return [p.strip() for p in re.split(r'[;|]', s) if p.strip()]

def commons_thumb_html(value: str, width: int = 300) -> str:
    """
    Build a clickable thumbnail for a Commons file, or a link if it's a Category.
    Accepts 'File:...', 'Image:...', 'Media:...', 'Category:...', or bare filename.
    """
    v = value.strip()
    lower = v.lower()
    if lower.startswith("category:"):
        url = f"https://commons.wikimedia.org/wiki/{quote(v.replace(' ', '_'))}"
        return f'<a href="{url}" target="_blank">{v}</a>'

    # Extract filename
    if any(lower.startswith(p) for p in ("file:", "image:", "media:")):
        filename = v.split(":", 1)[1]
    else:
        filename = v
    fn_enc = quote(filename.replace(" ", "_"))
    img = f"https://commons.wikimedia.org/wiki/Special:FilePath/{fn_enc}?width={width}"
    page = f"https://commons.wikimedia.org/wiki/File:{fn_enc}"
    return f'<a href="{page}" target="_blank"><img src="{img}" style="max-width:{width}px"></a>'

def link_commons_title(value: str) -> str:
    """Link any Commons title (File:/Category:/Gallery) to its page."""
    url = f"https://commons.wikimedia.org/wiki/{quote(value.replace(' ', '_'))}"
    return f'<a href="{url}" target="_blank">{value}</a>'

def link_wikidata(qid: str) -> str:
    """Link a Wikidata Q-id if it looks like one, else return raw."""
    if not qid:
        return ""
    q = qid.strip().upper()
    if q.startswith("Q") and q[1:].isdigit():
        return f'<a href="https://www.wikidata.org/wiki/{q}" target="_blank">{q}</a>'
    return q

def images_html_from_tags(tags: dict, max_thumbs: int = 2, thumb_width: int = 300) -> str:
    """
    Collect images from:
      - image, image:0, image:1, ... (absolute URLs or Commons titles)
      - wikimedia_commons (File:/Category:/Gallery) — supports multiple separated by ; or |
    Returns HTML for up to max_thumbs thumbnails/links.
    """
    candidates = []

    # image + image:* keys
    for k, v in tags.items():
        if not v:
            continue
        if k == "image" or k.startswith("image:"):
            candidates.append(v)

    # wikimedia_commons may contain multiple entries
    wc = tags.get("wikimedia_commons")
    if wc:
        candidates.extend(_split_multi(wc))

    html_parts = []
    for val in candidates:
        val = val.strip()
        if not val:
            continue
        if val.startswith("http://") or val.startswith("https://"):
            # Direct URL (may be an image or a page)
            html_parts.append(f'<a href="{val}" target="_blank"><img src="{val}" style="max-width:{thumb_width}px"></a>')
        else:
            # Assume Commons title
            html_parts.append(commons_thumb_html(val, width=thumb_width))

        if len(html_parts) >= max_thumbs:
            break

    return "<br>".join(html_parts)

drinkingwaters_fg = FeatureGroup(name="Dricksvatten inom 200 m")
for _, r in joined.to_crs(4326).iterrows():
    tags = r.get("tags", {}) or {}

    # Build OSM object link
    osm_path = ('node' if r['osm_type'] == 'node'
                else 'way' if r['osm_type'] == 'way'
                else 'relation')
    osm_url = f"https://www.openstreetmap.org/{osm_path}/{r['id']}"

    # Pictures (image*, wikimedia_commons)
    pics_html = images_html_from_tags(tags, max_thumbs=2, thumb_width=300)

    # Notes
    note_sv = tags.get("note")
    note_en = tags.get("note:en")

    # Commons links (all entries, not just thumbs)
    commons_entries = []
    wc = tags.get("wikimedia_commons")
    if wc:
        commons_entries = _split_multi(wc)
    commons_links = ", ".join(link_commons_title(x) for x in commons_entries) if commons_entries else ""

    # Operator + Wikidata
    operator = tags.get("operator")
    operator_wd = link_wikidata(tags.get("operator:wikidata", ""))

    # Access
    access = tags.get("access")

    # Popup HTML
    popup_html = f"""
    <b><a href="{osm_url}" target="_blank">OSM-objekt ({r['osm_type']})</a></b><br>
    Etapp: <b>{r.get('label','')}</b> (Ö: {r.get('island','')})<br>
    Avstånd: ~{round(r.get('distance_m', 0) or 0, 1)} m<br>
    """

    if pics_html:
        popup_html += pics_html + "<br>"

    if note_sv:
        popup_html += f"Not: {note_sv}<br>"
    if note_en:
        popup_html += f"Note (en): {note_en}<br>"

    if commons_links:
        popup_html += f"Wikimedia Commons: {commons_links}<br>"

    if operator:
        popup_html += f"Operatör: {operator}<br>"
    if operator_wd:
        popup_html += f"Operatör (Wikidata): {operator_wd}<br>"

    if access:
        popup_html += f"Access: {access}<br>"

    Marker(
        location=[r.geometry.y, r.geometry.x],
        popup=Popup(popup_html, max_width=360),
        icon=Icon(color="green", icon="tint")
    ).add_to(drinkingwaters_fg)

drinkingwaters_fg.add_to(m)
LayerControl(collapsed=False).add_to(m)

map_html = f"../kartor/Issue_139_dricksvatten_nara_stockholm_archipelago_trail_{timestamp}.html"
m.save(map_html)

print("✅ Klart!")
print(f"• Summary CSV: {summary_csv}")
print(f"• Dricksvatten GeoJSON: {waters_geojson}")
print(f"• Dricksvatten CSV: {waters_csv}")
print(f"• Karta: {map_html}")


🔍 Hämtar SAT-etapper från Wikidata...
✅ Hittade 20 etapper med OSM-relationer
📡 Hämtar geometrier från Overpass (per relation)...
🧭 Etapper med geometri: 20
🧮 Skapar 200 m-buffert...
💧 Hämtar dricksvatten (nwr) från Overpass...
✅ Hittade 132 dricksvatten-objekt inom bbox
✅ 37 dricksvatten-objekt inom/vid 200 m
📊 Summary drinking water:
            label     island  sites  avg_distance_m
1    SAT Finnhamn   Finnhamn      7       29.926556
11       SAT Rånö       Rånö      4       10.268689
14        SAT Utö        Utö      4       41.698753
0     SAT Arholma    Arholma      3       65.324714
15        SAT Ålö        Ålö      3       40.956089
5    SAT Landsort        Öja      2       20.177926
6        SAT Lidö       Lidö      2       37.572461
8       SAT Nämdö      Nämdö      2       11.054254
9     SAT Nåttarö    Nåttarö      2       75.400097
10       SAT Ornö       Ornö      2       46.438390
2   SAT Fjärdlång  Fjärdlång      1        5.464993
3      SAT Grinda     Grinda      1   